# Solutions to exercises on morphology

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.morphology as morph
import skimage.transform as trans
from skimage import measure
from scipy import ndimage

%matplotlib inline

## Basic operations
Test the effect of erosion, dilation, open, close on the test
image sand_bilevel.png. 

### Exercise 1
Change the size of the structure elements. Hint: use a for-loop and plt.subplot

In [ ]:
img=plt.imread('sand_bilevel.png')
plt.imshow(img)
N=[3,5,7]

#### Erode

In [ ]:
# your code here
plt.figure(figsize=[12,4])
for idx,n in enumerate(N) :
    plt.subplot(1,len(N),idx+1)
    plt.imshow(morph.erosion(img,np.ones([n,n])))

#### Dilate

In [ ]:
# your code here
plt.figure(figsize=[12,4])
for idx,n in enumerate(N) :
    plt.subplot(1,len(N),idx+1)
    plt.imshow(morph.dilation(img,np.ones([n,n])))

#### Open

In [ ]:
# your code here
plt.figure(figsize=[12,4])
for idx,n in enumerate(N) :
    plt.subplot(1,len(N),idx+1)
    plt.imshow(morph.opening(img,np.ones([n,n])))

#### Close

In [ ]:
# your code here
plt.figure(figsize=[12,4])
for idx,n in enumerate(N) :
    plt.subplot(1,len(N),idx+1)
    plt.imshow(morph.closing(img,np.ones([n,n])))

__Q1.1__: What can happen when the structure element is too large for
1. Open
2. Close
    
__Q1.2__: Which morphologic operation would you use to remove misclassified pixels in...
1. the pores
2. the grains

## Distance maps

### Exercise 2

Compute distance maps of the grains in the sand image using the
three metrics Euclidean, City-block, and Chessboard. Compute difference images between the first one
and the other two. 

In [ ]:
# your code here
euclid = ndimage.distance_transform_edt(img)
chessd = ndimage.morphology.distance_transform_cdt(img, metric='chessboard')
cityd  = ndimage.morphology.distance_transform_cdt(img, metric='taxicab')
plt.figure(figsize=[12,8])
plt.subplot(2,3,1)
plt.imshow(euclid)
plt.title('Euclidean distance')

plt.subplot(2,3,2)
plt.imshow(chessd)
plt.title('Chessboard distance')
plt.subplot(2,3,5)
plt.imshow(chessd-euclid)

plt.subplot(2,3,3)
plt.imshow(cityd)
plt.title('City-block distance')
plt.subplot(2,3,6)
plt.imshow(cityd-euclid)




__Q2.1__: What conclusion can you make regarding the
accuracy of the two connectivity based metrics compared to the
Euclidean metric?

Keep the result of the Euclidean distance map for the next exercises.

### Answer: 

__Q2.1__: Chess board tend to under estimate the distance while city-block rather over estimate the distance.

## Local extrema

Compute the distance maps of both grains and pore space in the
sand image and combine them into one image as

$g=D_{\mathcal{E}}(f^c)-D_{\mathcal{E}}(f)$

In [ ]:
# your code here
g =  ndimage.distance_transform_edt(1-img) - ndimage.distance_transform_edt(img)
plt.imshow(g)

### Exercise 3 
Test the palette of local extrema functions (emin,emax,hmin,hmax) on $g$.

__Q3.1__: What happens when you set $h$=5 resp. $h$=10?

__Q3.2__: Why are there more maxima than minima for these h values?

In [ ]:
# your code here
hh=[5,10]
plt.figure(figsize=(12,8))
for idx,h in enumerate(hh) :
    plt.subplot(len(hh),4,1+idx*4)
    plt.imshow(morph.local_minima(g))
    plt.title('EMin')
    plt.subplot(len(hh),4,2+idx*4)
    plt.imshow(morph.local_maxima(g))
    plt.title('EMax')
    plt.subplot(len(hh),4,3+idx*4)
    plt.imshow(morph.h_minima(g,h=h))
    plt.title('hMin h={0}'.format(h))
    plt.subplot(len(hh),4,4+idx*4)
    plt.imshow(morph.h_maxima(g,h=h))
    plt.title('hMin h={0}'.format(h))
    

### Answers

__Q3.1__: Less extrema are found when h increases because there are less locations that fit a disc with radius 10 within the item.

__Q3.2__: The pore space is more complex and provides more local extrema.

## Watershed segmentation
Reading: http://scikit-image.org/docs/dev/api/skimage.morphology.html?highlight=watershed#skimage.morphology.watershed
and 
http://www.scipy-lectures.org/packages/scikit-image/auto_examples/plot_segmentations.html

### Exercise 4
To segment a binary image with WS you have to compute it's
distance map (hopefully already done as image $g$ in exercise 4). You also need a marker image that tells the algorithm where to start. I suggest using h_maxima to find the markers.

Show the labeled image with a colormap that makes it possible to identify neighbors. This can be hard with the default colormap. 

__Q4__: What happens when you change the value of $h$ when you create the marker image?

In [ ]:
distance = ndimage.distance_transform_edt(img)
h=3 # what happens when h is changed?
plt.figure(figsize=[15,4])
plt.subplot(1,3,1)
plt.imshow(img)
plt.title('Bilevel input image')
plt.subplot(1,3,2)
local_maxi=morph.h_maxima(distance,h)
markers = measure.label(local_maxi)
plt.imshow(markers)
plt.title('Markers')
plt.subplot(1,3,3)
labels_ws = morph.watershed(-distance, markers, mask=img)
plt.imshow(labels_ws,cmap=plt.get_cmap('tab20'))
plt.title('Watershed segmented grains')

### Answer

__Q4__: Small values of h result over segmentation and larger values of h under segmentation.